# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger', 'omw-1.4'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterClean.db')
df = pd.read_sql("SELECT * FROM Msg_Category_Tbl", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# get input (X) / output (Y) data
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process  text data

In [4]:
def tokenize(text):
    """Process text messages: Lemmatize, lower, remove blank, remove stop words
    
    inputs:
        text (String): text messages
       
    outputs:
        clean_tokens (list): clean tokens
    """
    
    # get list of stopwords in english
    stop_words = stopwords.words("english")
    
    # tokenize words
    tokens = word_tokenize(text)
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    
    # result
    clean_tokens = []
    for token in tokens:
        # Lemmatize + lower + remove blank
        clean = lemmatizer.lemmatize(token).lower().strip()
        # do not add stop word to result
        if clean not in stop_words:
            clean_tokens.append(clean)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset.

In [5]:
# create pipline with TF-IDF + RandomForestClassifier
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

In [7]:
# train
pipe.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test  model
Report the f1 score, precision and recall for each output category of the dataset.

In [8]:
# get columns name as list
col_names = Y_train.columns.tolist()

In [9]:
# report the f1 score, precision and recall for each output category of the dataset using classification_report
y_pred = pipe.predict(X_test)
report1 = classification_report(Y_test, y_pred, target_names = col_names)
print(report1)

                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      4967
               request       0.84      0.50      0.62      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.74      0.67      0.70      2679
          medical_help       0.55      0.06      0.11       519
      medical_products       0.82      0.10      0.18       321
     search_and_rescue       0.75      0.02      0.04       153
              security       0.00      0.00      0.00       124
              military       0.67      0.07      0.13       223
                 water       0.86      0.31      0.45       416
                  food       0.82      0.59      0.69       724
               shelter       0.81      0.38      0.52       579
              clothing       0.75      0.06      0.11        97
                 money       0.62      0.04      0.07       141
        missing_people       0.00      

C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve model
Use grid search to find better parameters. 

In [24]:
# get parameters of pipe
pipe.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [39]:
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
              'clf__estimator__n_estimators':[50, 100, 150], 
              'clf__estimator__min_samples_split':[2, 5, 7]}

cv = GridSearchCV(estimator=pipe, param_grid=parameters, scoring='f1_weighted', verbose=3, cv=3)

### 7. Test  model
Show the accuracy, precision, and recall of the tuned model.  

In [40]:
# train
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.557 total time= 3.9min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.560 total time= 3.6min
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.548 total time= 3.6min
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.542 total time= 6.7min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.542 total time= 6.1min
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.532 total time= 6.5min
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngr

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5, 7],
                         'clf__estimator__n_estimators': [50, 100, 150],
                         'vect__ngram_range': ((1, 1), (1, 2))},
             scoring='f1_weighted', verbose=3)

In [41]:
cv.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 50,
 'vect__ngram_range': (1, 1)}

In [42]:
# predict and show report
y_pred2 = cv.predict(X_test)
report2 = classification_report(Y_test, y_pred2, target_names = col_names)
print(report2)

                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      4967
               request       0.83      0.51      0.63      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.73      0.67      0.70      2679
          medical_help       0.59      0.10      0.17       519
      medical_products       0.82      0.14      0.24       321
     search_and_rescue       1.00      0.02      0.04       153
              security       0.00      0.00      0.00       124
              military       0.67      0.07      0.13       223
                 water       0.84      0.32      0.47       416
                  food       0.82      0.54      0.65       724
               shelter       0.80      0.38      0.51       579
              clothing       0.75      0.06      0.11        97
                 money       0.67      0.04      0.08       141
        missing_people       1.00      

C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving  model further

Try to use other argothrims

In [43]:
# AdaBoostClassifier
pipe_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [44]:
# SGDClassifier
pipe_sgdc = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)))
    ])

In [46]:
# i will try to use other methods to improve my model
# so i create this fuction to reuse many times
# train and create test report
def train_test_model(pipe, x_train, y_train, x_test, y_test):
    """train model and create test report using classification_report
    
    inputs:
        pipe: model
        x_train, y_train: train data
        x_test, y_test: test data
       
    outputs:
        report: report generated by classification_report
    """
    # train
    pipe.fit(x_train, y_train)
    
    # predict
    y_pred = pipe.predict(x_test)
    
    # columns name
    col_names_list = y_train.columns.tolist()
    
    report = classification_report(y_test, y_pred, target_names = col_names_list)
    
    return report

In [47]:
# train + test AdaBoostClassifier
report_ada = train_test_model(pipe_ada, X_train, Y_train, X_test, Y_test)
print(report_ada)

                        precision    recall  f1-score   support

               related       0.82      0.92      0.87      4967
               request       0.76      0.52      0.62      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.74      0.59      0.66      2679
          medical_help       0.58      0.23      0.33       519
      medical_products       0.62      0.33      0.43       321
     search_and_rescue       0.45      0.13      0.20       153
              security       0.19      0.02      0.04       124
              military       0.60      0.33      0.43       223
                 water       0.76      0.66      0.71       416
                  food       0.80      0.67      0.73       724
               shelter       0.75      0.54      0.63       579
              clothing       0.66      0.38      0.48        97
                 money       0.45      0.26      0.33       141
        missing_people       0.48      

C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
# train + test SGDClassifier
report_sgdc = train_test_model(pipe_sgdc, X_train, Y_train, X_test, Y_test)
print(report_sgdc)

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      4967
               request       0.87      0.35      0.50      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.81      0.43      0.56      2679
          medical_help       0.00      0.00      0.00       519
      medical_products       0.00      0.00      0.00       321
     search_and_rescue       0.00      0.00      0.00       153
              security       0.00      0.00      0.00       124
              military       0.00      0.00      0.00       223
                 water       0.86      0.09      0.17       416
                  food       0.80      0.45      0.58       724
               shelter       0.88      0.19      0.32       579
              clothing       0.00      0.00      0.00        97
                 money       0.00      0.00      0.00       141
        missing_people       0.00      

C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AdaBoostClassifier appears to be better suited to my model.
So i will stick with AdaBoostClassifier.

Try to tunning model

In [49]:
# get parameters of pipe
pipe_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000019ABEC091F0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [50]:
parameters_ada = {'vect__ngram_range': ((1, 1), (1, 2)),
                  'clf__estimator__learning_rate': [0.5, 1.0, 1.5, 2.0],
                  'clf__estimator__n_estimators':[25, 50, 75]}

cv_ada = GridSearchCV(estimator=pipe_ada, param_grid=parameters_ada, scoring='f1_weighted', verbose=3, cv=3)

In [51]:
report_cv_ada = train_test_model(cv_ada, X_train, Y_train, X_test, Y_test)
print(report_cv_ada)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 1);, score=0.552 total time=  38.5s
[CV 2/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 1);, score=0.562 total time=  38.1s
[CV 3/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 1);, score=0.552 total time=  40.6s
[CV 1/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 2);, score=0.559 total time= 1.3min
[CV 2/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 2);, score=0.563 total time= 1.2min
[CV 3/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=25, vect__ngram_range=(1, 2);, score=0.558 total time= 1.3min
[CV 1/3] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1)

[CV 3/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=25, vect__ngram_range=(1, 2);, score=0.461 total time= 1.3min
[CV 1/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.209 total time= 1.1min
[CV 2/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.211 total time= 1.1min
[CV 3/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 1);, score=0.208 total time= 1.1min
[CV 1/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.201 total time= 2.4min
[CV 2/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.207 total time= 2.5min
[CV 3/3] END clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=50, vect__ngram_range=(1, 2);, score=0.203 total time= 2.5min
[CV 1/3] END clf__estimator

C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucky\anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
cv_ada.best_params_

{'clf__estimator__learning_rate': 1.0,
 'clf__estimator__n_estimators': 75,
 'vect__ngram_range': (1, 2)}

### 9. Export  model as a pickle file

In [ ]:
pickle.dump(cv_ada, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`